# Cеминар 10. Выборочные моменты ООП и код-стиль.

На этом, заключительном семинаре, мы обсудим некоторые разрозненные полезные факты про ООП, а закончим обсуждением код-стиля.

**ООП**

1. Связанные методы
2. Наследование от встроенных типов
3. Переменные класса и методы класса
4. Свойства

**Код-стиль**

1. Отступы
2. Длина строки
3. Пробелы

### Intro

Создадим класс, на основе которого мы будем рассматривать наши примеры.

In [2]:
class Account:
    '''
    Простой банковский счет
    '''
    owner: str
    balance: float
    
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance
        
    def __repr__(self):
        return f'Account({self.owner!r}, {self.balance!r})'
    
    def deposit(self, amount):
        self.balance += amount
        
    def withdraw(self, amount):
        self.balance -= amount
        
    def inquiry(self):
        return self.balance

In [4]:
a = Account('Guido', 1000.0)
# Вызывает Account.__init__(a, 'Guido', 1000.0)

b = Account('Eva', 10.0)
# Вызывает Account.__init__(b, 'Eva', 10.0)

У каждого экземпляра свое состояние. Для просмотра переменных экземпляров есть функция `vars()`:

In [5]:
a = Account('Guido', 1000.0)
b = Account('Eva', 10.0)
vars(a)

{'owner': 'Guido', 'balance': 1000.0}

In [6]:
vars(b)

{'owner': 'Eva', 'balance': 10.0}

### Связанные методы

Вместо выполнения операций оператором «точка» (`.`) можно передать имя атрибута в строковом виде функциям `getattr()`, `setattr()` и `delattr()`. 

Функция `hasattr()` проверяет наличие атрибута:

In [7]:
a = Account('Guido', 1000.0)
getattr(a, 'owner')

'Guido'

In [8]:
setattr(a, 'balance', 750.0)
delattr(a, 'balance')
hasattr(a, 'balance')

False

In [9]:
getattr(a, 'withdraw')(100) # Вызов метода
a

AttributeError: 'Account' object has no attribute 'balance'

Обращаясь к методу как к атрибуту, вы получаете объект, называемый связанным методом:

In [10]:
a = Account('Guido', 1000.0)
w = a.withdraw
w

<bound method Account.withdraw of Account('Guido', 1000.0)>

In [11]:
w(100)
a

Account('Guido', 900.0)

Связанный метод — это объект, содержащий как экземпляр (`self`), так и функцию, реализующую метод. 

Когда вы вызываете связанный метод, добавляя круглые скобки и аргументы, он выполняет метод, передавая присоединенный экземпляр в качестве первого аргумента. 

Например, вызов `w(100)` преобразуется в `Account.withdraw(a, 100)`

### Наследование от встроенных типов

Python допускает наследование от встроенных типов. Но оно сопряжено с определенным риском. Например, если вы решили субклассировать `dict`, чтобы принудительно использовать ключи в верхнем регистре, можно переопределить метод `__setitem__()` так:

In [12]:
class udict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key.upper(), value)

И на первый взгляд такое решение работает:

In [13]:
u = udict()
u['name'] = 'Guido'
u['number'] = 37
u

{'NAME': 'Guido', 'NUMBER': 37}

Но потом выясняется, что это не так — вам только казалось, что класс работает. И теперь начинает казаться, что он вообще не работает:

In [14]:
u.update(color='blue')
u

{'NAME': 'Guido', 'NUMBER': 37, 'color': 'blue'}

Проблема в том, что встроенные типы Python не реализуются как нормальные классы Python — они написаны на C. Большинство методов работают в мире C. Например, `dict.update()` напрямую манипулирует данными словаря в обход переопределенного метода `__setitem__()` из вашего класса `udict`.

В модуле `collections` есть специальные классы `UserDict`, `UserList` и `UserString`. Они могут использоваться для создания безопасных субклассов `dict`, `list` и `str`. Следующее решение работает гораздо лучше:

In [15]:
from collections import UserDict

class udict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key.upper(), value)

Пример использования новой версии:

In [16]:
u = udict(name='Guido', num=37)
u.update(color='Blue')
u

{'NAME': 'Guido', 'NUM': 37, 'COLOR': 'Blue'}

In [17]:
v = udict(u)
v['title'] = 'Seminar'
v

{'NAME': 'Guido', 'NUM': 37, 'COLOR': 'Blue', 'TITLE': 'Seminar'}

Обычно субклассирования встроенных типов лучше избегать. Если вам действительно нужно субклассировать встроенный тип, то приготовьтесь, что работы будет больше, чем вам казалось изначально.

### Переменные класса и методы класса

Вообще говоря, пример использования - счетчик открытых счетов

In [20]:
class Account:
    num_accounts = 0
    
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance
        Account.num_accounts += 1

Переменные класса определяются за пределами обычного метода `__init__()`. Для их изменения нужно использовать класс, а не `self`:

In [21]:
a = Account('Guido', 1000.0)
b = Account('Eva', 10.0)
Account.num_accounts

2

К переменным класса можно обращаться и через экземпляры, хотя это и немного необычно:

In [22]:
a.num_accounts

2

### @classmethod

Также возможно определить и методы класса. Они применяются к самому классу, а не к экземплярам. Обычно методы класса используются для определения альтернативных конструкторов экземпляров. Представьте, что в требованиях была оговорена возможность создания экземпляров Account по унаследованному формату данных корпоративного уровня:

In [ ]:
data = '''
<account>
<owner>Guido</owner>
<amount>1000.0</amount>
</account>
'''

Для этого можно написать метод класса:

In [ ]:
class Account:
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance
        
    @classmethod
    def from_xml(cls, data):
        from xml.etree.ElementTree import XML
        doc = XML(data)
        return cls(doc.findtext('owner'), float(doc.findtext('amount')))
    
# Пример использования
data = '''
<account>
<owner>Guido</owner>
<amount>1000.0</amount>
</account>
'''
a = Account.from_xml(data)

В первом аргументе метода класса всегда передается сам класс. Этот аргумент часто называется `cls`. Здесь `cls` присваивается `Account`. Если цель метода класса - создание нового экземпляра, для этого должны быть предприняты явные шаги. В последней строке примера вызов `cls(..., ...)` аналогичен вызову `Account(..., ...)` с двумя аргументами.

Альтернативное конструирование экземпляров - самое частое применение методов классов. В популярной схеме выбора имен таких методов используется префикс `from_`, например `from_timestamp()`. Эта схема встречается в методах класса в стандартной библиотеке и в сторонних пакетах.

### @staticmethod

Иногда класс просто используется как пространство имен для функций, объявленных как статические методы с использованием `@staticmethod`. В отличие от обычного метода или метода класса, статический не получает дополнительный аргумент `self` или `cls`. Это обычная функция, которая определяется внутри класса:

In [23]:
class Ops:
    @staticmethod
    def add(x, y):
        return x + y
    
    @staticmethod
    def sub(x, y):
        return x - y

Экземпляры таких классов обычно не создаются. Вместо этого используется прямой вызов функций через класс:

In [25]:
a = Ops.add(2, 3) # a = 5
b = Ops.sub(4, 5) # a = -1
a, b

(5, -1)

### Свойства

По умолчанию Python не устанавливает никаких ограничений времени выполнения для значений или типов атрибутов. Но такие ограничения возможны. Для этого нужно поместить атрибут под управление свойства. Это разновидность атрибута, которая перехватывает обращения к нему и обрабатывает их методами, определенными пользователем. Такие методы могут управлять атрибутом так, как считают нужным:

In [26]:
import string

class Account:
    def __init__(self, owner, balance):
        self.owner = owner
        self._balance = balance
        
    @property
    def owner(self):
        return self._owner
    
    @owner.setter
    def owner(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected str')
        if not all(c in string.ascii_uppercase for c in value):
            raise ValueError('Must be uppercase ASCII')
        if len(value) > 10:
            raise ValueError('Must be 10 characters or less')
            self._owner = value

Здесь атрибут owner ограничивается строкой из 10 символов верхнего регистра в кодировке ASCII. Вот, как это работает при использовании класса:

In [28]:
a = Account('GUIDO', 1000.0)
a.owner = 'EVA'

In [29]:
a.owner = 42

TypeError: Expected str

In [30]:
a.owner = 'Carol'

ValueError: Must be uppercase ASCII

In [31]:
a.owner = 'RENÉE'

ValueError: Must be uppercase ASCII

In [32]:
a.owner = 'RAMAKRISHNAN'

ValueError: Must be 10 characters or less

`@property` помечает атрибут как свойство. Здесь он применяется к атрибуту `owner`. Этот декоратор всегда применяется к методу, получающему значение атрибута. В этом примере метод возвращает фактическое значение, которое сохраняется в приватном атрибуте `_owner`. Декоратор `@owner.setter` используется для необязательной реализации метода, присваивающего значение атрибута. Этот метод выполняет проверки типа и значения перед сохранением значения в `_owner`.

Важнейшая особенность свойств в том, что связанное с ними имя (как `owner` в этом примере) становится «волшебным»: любое использование этого атрибута автоматически направляется через реализованные вами методы чтения/записи. Вам не придется изменять код, чтобы эта схема заработала. Не нужно вносить изменения в метод `Account.__init__()`. 

Вас это может удивить, ведь `__init__()` выполняет присваивание `self.owner = owner` вместо использования приватного атрибута `self._owner`. Это сделано специально: свойство owner вводилось именно для проверки значений атрибута, что определенно нужно делать при создании экземпляров. Вы увидите, что все работает как предполагалось:

In [33]:
a = Account('Guido', 1000.0)

ValueError: Must be uppercase ASCII

При каждом обращении к атрибуту свойства автоматически вызывается метод, поэтому реальное значение должно храниться под другим именем. Вот почему внутри методов чтения и записи используется имя _owner. owner не может использоваться для хранения - это приведет к бесконечной рекурсии.
Как правило, свойства позволяют перехватывать любое конкретное имя атрибута. Вы можете реализовать методы для чтения, записи или удаления значения атрибута:

In [34]:
class SomeClass:
    @property
    def attr(self):
        print('Getting')
        
    @attr.setter
    def attr(self, value):
        print('Setting', value)
        
    @attr.deleter
    def attr(self):
        print('Deleting')

In [35]:
s = SomeClass()
s.attr # Чтение
s.attr = 13 # Запись
del s.attr # Удаление

Getting
Setting 13
Deleting


Реализовать все части свойства необязательно. На самом деле свойства часто используются для реализации атрибутов вычисляемых данных, доступных только для чтения:

In [37]:
class Box(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    @property
    def area(self):
        return self.width * self.height
    
    @property
    def perimeter(self):
        return 2*self.width + 2*self.height
    
# Пример использования
b = Box(4, 5)
print(b.area) # -> 20
print(b.perimeter) # -> 18

20
18


In [38]:
b.area = 5 # Ошибка: невозможно задать атрибут

AttributeError: can't set attribute

## Код-стиль

### Паттерны - word of caution

При написании объектно-ориентированных программ многие стремятся реализовать известные паттерны проектирования: «стратегия», «приспособленец», «одиночка» и т. д. Большинство из них происходят из знаменитой книги «Паттерны проектирования» Эрика Гаммы, Ричарда Хелма, Ральфа Джонсона и Джона Влиссидеса. Если вам знакомы эти паттерны, общие принципы проектирования в других языках могут быть применены и в Python. Но многие документированные шаблоны предназначены для решения конкретных проблем, возникающих из-за строгой статической системы типов C++ или Java. Динамическая природа Python делает многие из этих шаблонов устаревшими, излишними или просто ненужными.

Есть ряд универсальных принципов написания хорошего кода. Например, стремление писать код простой в отладке, тестировании и расширении. Такие базовые стратегии, как написание классов с полезными методами `__repr__()`, предпочтение композиции перед наследованием

Программисты Python предпочитают работать с кодом, который называют питоническим. Это значит, что объекты соблюдают разные встроенные протоколы (перебор, контейнеры или управление контекстом). Например, вместо того чтобы пытаться реализовать какой-то экзотический шаблон обхода данных из книги по программированию на Java, программист Python реализует его с помощью функции-генератора или просто заменит весь шаблон несколькими поисками по словарю.

### PEP-8 

**Лейтмотив - код читают гораздо чаще, чем пишут!**

### Отступы

**НЕ**правильно:

In [ ]:
# Arguments on first line forbidden when not using vertical alignment.
foo = long_function_name(var_one, var_two,
    var_three, var_four)

# Further indentation required as indentation is not distinguishable.
def long_function_name(
    var_one, var_two, var_three,
    var_four):
    print(var_one)

Правильно:

In [ ]:
# Aligned with opening delimiter.
foo = long_function_name(var_one, var_two,
                         var_three, var_four)

# Add 4 spaces (an extra level of indentation) to distinguish arguments from the rest.
def long_function_name(
        var_one, var_two, var_three,
        var_four):
    print(var_one)

# Hanging indents should add a level.
foo = long_function_name(
    var_one, var_two,
    var_three, var_four)

Опционально:

In [ ]:
# Hanging indents *may* be indented to other than 4 spaces.
foo = long_function_name(
  var_one, var_two,
  var_three, var_four)

If-конструкции

In [ ]:
# No extra indentation.
if (this_is_one_thing and
    that_is_another_thing):
    do_something()

# Add a comment, which will provide some distinction in editors
# supporting syntax highlighting.
if (this_is_one_thing and
    that_is_another_thing):
    # Since both conditions are true, we can frobnicate.
    do_something()

# Add some extra indentation on the conditional continuation line.
if (this_is_one_thing
        and that_is_another_thing):
    do_something()

Закрывающая скобка может располагаться так:

In [ ]:
my_list = [
    1, 2, 3,
    4, 5, 6,
    ]
result = some_function_that_takes_arguments(
    'a', 'b', 'c',
    'd', 'e', 'f',
    )

Или так

In [ ]:
my_list = [
    1, 2, 3,
    4, 5, 6,
]
result = some_function_that_takes_arguments(
    'a', 'b', 'c',
    'd', 'e', 'f',
)

### Длина строки

Максимальная длина строки - 79 символов (по договоренности внутри команды можно 99, но не более) , для докстринга и комментариев - 72 символа.

Лучше всего обходиться механизмами, позволяющими объединить строки по умолчанию (такими, как скобки) - должно иметь приоритет перед `\`, хоть иногда использование последнего и оправдано (например, с контекст-менеджером `with` или `assert`):

In [ ]:
with open('/path/to/some/file/you/want/to/read') as file_1, \
     open('/path/to/some/file/being/written', 'w') as file_2:
    file_2.write(file_1.read())

Разрыв строки перед бинарным оператором.

**НЕ**правильно:

In [ ]:
# operators sit far away from their operands
income = (gross_wages +
          taxable_interest +
          (dividends - qualified_dividends) -
          ira_deduction -
          student_loan_interest)

Правильно:

In [ ]:
# easy to match operators with operands
income = (gross_wages
          + taxable_interest
          + (dividends - qualified_dividends)
          - ira_deduction
          - student_loan_interest)

### Пробелы

Двоеточие не в роли оператора:

In [ ]:
# Correct:
if x == 4: print(x, y); x, y = y, x

In [ ]:
# Wrong:
if x == 4 : print(x , y) ; x , y = y , x

Двоеточие в роли оператора:

In [ ]:
# Correct:
ham[1:9], ham[1:9:3], ham[:9:3], ham[1::3], ham[1:9:]
ham[lower:upper], ham[lower:upper:], ham[lower::step]
ham[lower+offset : upper+offset]
ham[: upper_fn(x) : step_fn(x)], ham[:: step_fn(x)]
ham[lower + offset : upper + offset]

In [ ]:
# Wrong:
ham[lower + offset:upper + offset]
ham[1: 9], ham[1 :9], ham[1:9 :3]
ham[lower : : step]
ham[ : upper]

Бинарные операторы (=, +=, -=, ==, <, >, !=, <>, <=, >=, in, not in, is, is not, and, or, not) отделяются пробелом с каждой стороны. 

Если используется несколько операторов с разным приоритетом, пробел добавляется вокруг операторов с самым низким приоритетом. Но использовать разумно - никогда не использовать больше чем один пробел, и использовать одинаковое количество пробелов (ноль или один) по каждую сторону от бинарного оператора:

In [ ]:
# Correct:
i = i + 1
submitted += 1
x = x*2 - 1
hypot2 = x*x + y*y
c = (a+b) * (a-b)

In [ ]:
# Wrong:
i=i+1
submitted +=1
x = x * 2 - 1
hypot2 = x * x + y * y
c = (a + b) * (a - b)

In [ ]:
# Correct:
def munge(input: AnyStr): ...
def munge() -> PosInt: ...

In [ ]:
# Wrong:
def munge(input:AnyStr): ...
def munge()->PosInt: ...

Не используйте пробелы с `=` когда задаете именованный аргумент, или дефолтное значение:

In [ ]:
# Correct:
def complex(real, imag=0.0):
    return magic(r=real, i=imag)

In [ ]:
# Wrong:
def complex(real, imag = 0.0):
    return magic(r = real, i = imag)

Исключение - аннотация типов

In [ ]:
# Correct:
def munge(sep: AnyStr = None): ...
def munge(input: AnyStr, sep: AnyStr = None, limit=1000): ...

In [ ]:
# Wrong:
def munge(input: AnyStr=None): ...
def munge(input: AnyStr, limit = 1000): ...

### Комментарии и докстринг

In [ ]:
"""Return a foobang

Optional plotz says to frobnicate the bizbaz first.
"""

In [ ]:
"""Return an ex-parrot."""

# Спасибо за активное участие в курсе! 
# Успехов на дальнейших дисциплинах майнора!
# P.S. и карьере data scientist'а тоже ;)